<a href="https://colab.research.google.com/github/aryan802/food_recognition_and_nutritional_analysis/blob/main/deep_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Drive
drive.mount('/content/drive')

# Paths
zip_path = "/content/drive/MyDrive/archive (5).zip"  # change if in a different folder
extract_path = "/content/fast_food_data"

# Create folder and unzip
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted!


In [ ]:
train_dir = "/content/fast_food_data/Fast Food Classification V2/Train"
val_dir   = "/content/fast_food_data/Fast Food Classification V2/Valid"
test_dir  = "/content/fast_food_data/Fast Food Classification V2/Test"


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = val_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

num_classes = len(train_gen.class_indices)
print("Number of classes:", num_classes)


Found 15000 images belonging to 10 classes.
Found 3500 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.
Number of classes: 10


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze base model initially
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,062,381 (15.50 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, verbose=1)

callbacks = [checkpoint, early_stop, reduce_lr]

In [ ]:
initial_epochs = 10

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=initial_epochs,
    callbacks=callbacks
)

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step - accuracy: 0.5966 - loss: 1.2763
Epoch 1: val_accuracy improved from -inf to 0.79800, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 223s 434ms/step - accuracy: 0.5969 - loss: 1.2756 - val_accuracy: 0.7980 - val_loss: 0.6690 - learning_rate: 0.0010
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.8001 - loss: 0.6394
Epoch 2: val_accuracy improved from 0.79800 to 0.81829, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 181s 386ms/step - accuracy: 0.8001 - loss: 0.6393 - val_accuracy: 0.8183 - val_loss: 0.6113 - learning_rate: 0.0010
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.8190 - loss: 0.5778
Epoch 3: val_accuracy improved from 0.81829 to 0.82629, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 183s 389ms/step - accuracy: 0.8190 - loss: 0.5778 - val_accuracy: 0.8263 - val_loss: 0.5804 - learning_rate: 0.0010
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step - accuracy: 0.8279 - loss: 0.5455
Epoch 4: val_accuracy did not improve from 0.82629
469/469 ━━━━━━━━━━━━━━━━━━━━ 183s 390ms/step - accuracy: 0.8279 - loss: 0.5455 - val_accuracy: 0.8243 - val_loss: 0.5952 - learning_rate: 0.0010
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.8336 - loss: 0.5381
Epoch 5: val_accuracy improved from 0.82629 to 0.82800, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 182s 389ms/step - accuracy: 0.8336 - loss: 0.5380 - val_accuracy: 0.8280 - val_loss: 0.5793 - learning_rate: 0.0010
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.8370 - loss: 0.5086
Epoch 6: val_accuracy improved from 0.82800 to 0.83886, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 182s 388ms/step - accuracy: 0.8370 - loss: 0.5086 - val_accuracy: 0.8389 - val_loss: 0.5545 - learning_rate: 0.0010
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step - accuracy: 0.8357 - loss: 0.5182
Epoch 7: val_accuracy did not improve from 0.83886
469/469 ━━━━━━━━━━━━━━━━━━━━ 181s 387ms/step - accuracy: 0.8357 - loss: 0.5182 - val_accuracy: 0.8357 - val_loss: 0.5718 - learning_rate: 0.0010
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.8404 - loss: 0.4913
Epoch 8: val_accuracy did not improve from 0.83886
469/469 ━━━━━━━━━━━━━━━━━━━━ 180s 383ms/step - accuracy: 0.8404 - loss: 0.4913 - val_accuracy: 0.8323 - val_loss: 0.5781 - learning_rate: 0.0010
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.8435 - loss: 0.4987
Epoch 9: val_accuracy improved from 0.83886 to 0.84000, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 182s 388ms/step - accuracy: 0.8436 - loss: 0.4986 - val_accuracy: 0.8400 - val_loss: 0.5488 - learning_rate: 0.0010
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.8501 - loss: 0.4744
Epoch 10: val_accuracy did not improve from 0.84000
469/469 ━━━━━━━━━━━━━━━━━━━━ 182s 389ms/step - accuracy: 0.8501 - loss: 0.4744 - val_accuracy: 0.8377 - val_loss: 0.5594 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 9.


In [ ]:
test_loss, test_acc = model.evaluate(test_gen)
print("Test Accuracy:", test_acc)

47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.9004 - loss: 0.3483
Test Accuracy: 0.8546666502952576


In [ ]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Save the trained model to Drive
model_save_path = "/content/drive/MyDrive/fast_food_classifier.h5"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# -----------------------------------------------
# # To load the model later for inference, use:
# from tensorflow.keras.models import load_model
# model = load_model("/content/drive/MyDrive/fast_food_classifier.h5")
# -----------------------------------------------



Model saved to /content/drive/MyDrive/fast_food_classifier.h5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = "/content/drive/MyDrive/Fast Food Classification V2/Train"
valid_dir = "/content/drive/MyDrive/Fast Food Classification V2/Valid"
test_dir  = "/content/drive/MyDrive/Fast Food Classification V2/Test"
model_path = "/content/drive/MyDrive/fast_food_classifier.h5"

In [ ]:
zip_path = "/content/drive/MyDrive/archive (5).zip"  # Update with your Drive path
extract_path = "fast_food_data"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

# Check structure
!ls fast_food_data/Fast\ Food\ Classification\ V2/Train


'Baked Potato'	'Crispy Chicken'   Fries      Pizza	 Taco
 Burger		 Donut		  'Hot Dog'   Sandwich	 Taquito


In [7]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/fast_food_classifier.h5")

### DASHBOARD

In [2]:
# ===============================
# 🍔 FOOD RECOGNITION DASHBOARD (SIDE-BY-SIDE)
# Accurate, Non-scrollable, Beautiful Layout
# ===============================

# --- Install dependencies ---
!apt-get update -qq
!apt install -y nodejs npm -qq
!pip install -q streamlit tensorflow pandas matplotlib pillow
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || apt-get -f install -y -qq

import os, time
from google.colab import drive
drive.mount('/content/drive')

# --- EDIT THESE PATHS ---
MODEL_PATH = "/content/drive/MyDrive/fast_food_classifier.h5"
NUTRITION_CSV = "/content/drive/MyDrive/nutrition_data.csv"
DEMO_DIR = "/content/drive/MyDrive/demo_images"

# --- Write Streamlit App ---
app_code = r'''
import streamlit as st
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import os

# ============ Load Model & Data ============
@st.cache_resource
def load_model(path):
    return tf.keras.models.load_model(path)

@st.cache_data
def load_nutrition_data(path):
    try:
        return pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin1")

MODEL_PATH = "''' + MODEL_PATH + '''"
CSV_PATH = "''' + NUTRITION_CSV + '''"
DEMO_DIR = "''' + DEMO_DIR + '''"

model = load_model(MODEL_PATH)
nutrition_df = load_nutrition_data(CSV_PATH)

# Fixed class order (match training)
CLASS_NAMES = [
    "Baked Potato","Burger","Crispy Chicken","Donut","Fries",
    "Hot Dog","Pizza","Sandwich","Taco","Taquito"
]

# ============ Page Setup ============
st.set_page_config(page_title="🍔 Food Recognition Dashboard", layout="wide")
st.markdown("<h2 style='text-align:center;'>🍔 Food Recognition & Nutritional Analysis</h2>", unsafe_allow_html=True)
st.write("---")

# --- Create two columns: Left (input) | Right (results)
col1, col2 = st.columns([1, 1])

with col1:
    st.subheader("📸 Upload or Choose a Food Image")

    # Demo images
    demo_images = {}
    if os.path.exists(DEMO_DIR):
        for img_file in os.listdir(DEMO_DIR):
            if img_file.lower().endswith(("jpg","jpeg","png")):
                name = os.path.splitext(img_file)[0].replace("_", " ").title()
                demo_images[name] = os.path.join(DEMO_DIR, img_file)

    uploaded_file = st.file_uploader("Upload Image", type=["jpg","jpeg","png"])
    selected_demo = st.selectbox("Or choose a demo image", ["None"] + list(demo_images.keys()))

    if uploaded_file:
        st.image(uploaded_file, caption="Uploaded Image", use_container_width=True)
        img_src = uploaded_file
    elif selected_demo != "None":
        st.image(demo_images[selected_demo], caption=selected_demo, use_container_width=True)
        img_src = demo_images[selected_demo]
    else:
        st.image("https://cdn-icons-png.flaticon.com/512/857/857681.png",
                 caption="Upload or select an image to start", use_container_width=True)
        img_src = None

with col2:
    st.subheader("🔍 Prediction & Nutrition Insights")

    def predict_food(img_path_or_file):
        img = image.load_img(img_path_or_file, target_size=(224, 224))
        img_array = np.expand_dims(image.img_to_array(img), axis=0)
        img_array = preprocess_input(img_array)
        pred = model.predict(img_array)
        idx = np.argmax(pred)
        return CLASS_NAMES[idx], pred[0]

    if img_src:
        predicted_food, pred_probs = predict_food(img_src)
        st.markdown(f"<h3>🍽️ Predicted: <b>{predicted_food}</b></h3>", unsafe_allow_html=True)

        # Confidence table
        conf_df = pd.DataFrame({
            "Food Item": CLASS_NAMES,
            "Confidence (%)": [round(float(p)*100, 2) for p in pred_probs]
        }).sort_values("Confidence (%)", ascending=False).head(5)
        st.dataframe(conf_df, use_container_width=True, hide_index=True)

        # Nutrition info
        data = nutrition_df[nutrition_df["Food Item"].str.lower() == predicted_food.lower()]
        if not data.empty:
            st.markdown("### 🧾 Nutrition (per 100g)")
            st.dataframe(data[["Calories","Protein (g)","Fat (g)","Carbs (g)"]],
                         use_container_width=True, hide_index=True)
            st.success("💡 " + data["Tip"].values[0])

            # Pie chart
            fig, ax = plt.subplots(figsize=(3,3))
            ax.pie(
                [data["Protein (g)"].values[0],
                 data["Fat (g)"].values[0],
                 data["Carbs (g)"].values[0]],
                labels=["Protein","Fat","Carbs"],
                autopct='%1.1f%%', startangle=90
            )
            ax.axis("equal")
            st.pyplot(fig)
        else:
            st.warning("No nutrition data found for this food.")
    else:
        st.info("👈 Upload or select a demo image to begin analysis.")
'''

# Save and run Streamlit
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_code)

print("🚀 Launching Streamlit app (side-by-side layout)... Please wait 20–30 seconds.")
get_ipython().system_raw("streamlit run app.py --server.port 8501 &> /content/streamlit_log.txt &")

# --- Cloudflare Tunnel (no password / login) ---
print("🌐 Starting Cloudflare tunnel...")
time.sleep(5)
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
npm is already the newest version (8.5.1~ds-1).
nodejs is already the newest version (12.22.9~dfsg-1ubuntu3.6).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Launching Streamlit app (side-by-side layout)... Please wait 20–30 seconds.
🌐 Starting Cloudflare tunnel...
2025-11-02T08:22:25Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunne